## Spam Email Classifier with KNN using TF-IDF scores

1.   Assignment must be implemented in Python 3 only.
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for evaluation metrics, data visualization (matplotlib etc.).
3.   You will be evaluated not just on the overall performance of the model and also on the experimentation with hyper parameters, data prepossessing techniques etc.
4.   The report file must be a well documented jupyter notebook, explaining the experiments you have performed, evaluation metrics and corresponding code. The code must run and be able to reproduce the accuracies, figures/graphs etc.
5.   For all the questions, you must create a train-validation data split and test the hyperparameter tuning on the validation set. Your jupyter notebook must reflect the same.
6.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

**Task: Given an email, classify it as spam or ham**

Given input text file ("emails.txt") containing 5572 email messages, with each row having its corresponding label (spam/ham) attached to it.

This task also requires basic pre-processing of text (like removing stopwords, stemming/lemmatizing, replacing email_address with 'email-tag', etc..).

You are required to find the tf-idf scores for the given data and use them to perform KNN using Cosine Similarity.

### Import necessary libraries

In [3]:
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
import re
from sklearn.model_selection import train_test_split #using sklearn only for splitting data
from scipy import spatial

[nltk_data] Downloading package punkt to /home/kush/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### Load dataset

In [4]:
config = {
    "dataset_file":"./emails.txt", 
    "output_file": "./output_q3.txt",
    "split_ratio": 0.02
}

In [5]:
with open(config['dataset_file'], 'r') as inFile:
    dataset = inFile.readlines()
df = pd.DataFrame(dataset)

In [6]:
df[[0, 'string']] = df[0].str.split("\t", expand=True)
df = df.rename(columns = {0:'type','string': 'string'})

In [7]:
def one_hot(mail_type):
    if mail_type == "ham":
        return 0
    else: 
        return 1

In [8]:
df['type'] = df['type'].apply(one_hot) 

ham: 0,  spam: 1

### Preprocess data

In [9]:
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /home/kush/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/kush/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/kush/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [10]:
emails = df.to_numpy()

In [11]:

for i, string in enumerate(emails[:, 1]):
    string = re.sub(r"[^a-zA-Z0-9]", " ", string.lower()) #remove puncs and lower
    string = word_tokenize(string)                        #tokanize
    new_string = []
    for word in string:       
        if word not in stop_words:                        #remove stop_words
            word = lem.lemmatize(word)                    #Lemmatize                
            new_string.append(word)
    emails[i, 1] = new_string
    
    

In [12]:
texts = emails[:, 1]
labels = emails[:, 0]

In [13]:
def calc_Vocab(data):
    total_words = []
    for l in data:
        total_words.extend(l)
    total = list(set(total_words))
    return total    

In [14]:
def calc_tf(t, d):
    return d.count(t)/len(d)

In [15]:
def calc_df(tf):
    temp = tf.astype(bool).sum()
    df = dict(zip(list(tf), temp))
    return df
    

In [16]:
def calc_tfidf(data):
    vocab = calc_Vocab(data)
    tf = []
    N = len(data)
    for mail in data:
        word_dic = dict.fromkeys(vocab, 0)
        for word in mail:
            word_dic[word] = calc_tf(word, mail)
        tf.append(word_dic)
    tf = pd.DataFrame(tf)
    
    return tf
        

In [17]:
tf = calc_tfidf(texts)

In [18]:
N = len(texts)

In [19]:
tf_arr = tf.to_numpy()
df = calc_df(tf)   
df_arr = np.array(list(df.values()))
N_arr = np.full( shape=len(list(tf)), fill_value=N, dtype=np.int )
idf = np.log10(np.divide(N_arr, df_arr+1))

In [20]:
tfidf = tf_arr*idf

In [21]:
tfidf_df = pd.DataFrame(tfidf, columns= list(tf))

### Split data

In [22]:
train_data, val_data, train_label, val_label = train_test_split(tfidf, labels, test_size=config['split_ratio'], random_state = 37)

### Train your KNN model (reuse previously iplemented model built from scratch) and test on your data

In [23]:
#caculate cosine distance 


In [24]:
def get_cosine(a, b):
    return 1 - spatial.distance.cosine(a, b)

In [25]:
similarity_vec = []
for vec in val_data:
    cosin_1 = []
    for train_vec in train_data:
        cosin = get_cosine(train_vec, vec)
        cosin_1.append(cosin)
    similarity_vec.append(cosin_1)

/home/kush/.local/lib/python3.8/site-packages/scipy/spatial/distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


### TF-IDF Implementation

In [26]:
def neighbors(similarity,k_value):
    k_n_n = []
    for i in range(k_value):
        max_sim = 0
        max_sim_user = 0
        for i, sim in enumerate(similarity):
            if i in k_n_n:
                continue
            elif max_sim <= sim:
                max_sim_user = i
                max_sim = sim
        k_n_n.append(max_sim_user)
    return k_n_n

In [27]:
def pridict(knn, label):
    sumi = 0
    for n in knn:
        sumi += label[n] 
    if sumi > len(knn)/2:
        return 1
    else:
        return 0

In [28]:
pridictions = []
k = 5

In [29]:
for sim in similarity_vec:
    knn = neighbors(sim, k)
    pridictions.append(pridict(knn, train_label))

In [31]:
correct_prid = 0
for i in range(len(pridictions)):
    if pridictions[i] == val_label[i]:
        correct_prid += 1
print(correct_prid)
    

108


In [32]:
len(pridictions)

112

***1. Experiment with different distance measures [Euclidean distance, Manhattan distance, Hamming Distance] and compare with the Cosine Similarity distance results.***

***2. Explain which distance measure works best and why? Explore the distance measures and weigh their pro and cons in different application settings.***

***3. Report Mean Squared Error(MSE), Mean-Absolute-Error(MAE), R-squared (R2) score in a tabular form***

***4. Choose different K values (k=1,3,5,7,11,17,23,28) and experiment. Plot a graph showing R2 score vs k.***

### Train and test Sklearn's KNN classifier model on your data (use metric which gave best results on your experimentation with built-from-scratch model.)

***Compare both the models result.***

***What is the time complexity of training using KNN classifier?***

***What is the time complexity while testing? Is KNN a linear classifier or can it learn any boundary?***